In [38]:
import networkx as nx

import numpy as np
import pandas as pd
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
%matplotlib inline

#import community

import sklearn
from scipy.optimize import minimize
from sklearn.metrics import mean_squared_error #MSE
from sklearn.metrics import mean_absolute_error #MAE
from sklearn.metrics import r2_score#R 2


plt.rcParams['axes.unicode_minus']=False 

In [39]:
df = pd.read_csv("london_flows.csv")

In [40]:
G = nx.from_pandas_edgelist(df, 'station_origin', "station_destination", ["flows", "population",'jobs','distance'])

# I Topological network
## I.1 Centrality measures

In [4]:
def dict_sort_by_value(dict_input):
    return sorted(dict_input.items(),key=lambda x : x[1], reverse=True)  

In [5]:
betweenness_centrality_value = dict_sort_by_value(nx.betweenness_centrality(G))
closeness_centrality_value = dict_sort_by_value(nx.closeness_centrality(G))
eigenvector_centrality_value = dict_sort_by_value(nx.eigenvector_centrality(G))

In [6]:
df_betweenness_centrality = pd.DataFrame(betweenness_centrality_value)
df_closeness_centrality = pd.DataFrame(closeness_centrality_value)
df_eigenvector_centrality = pd.DataFrame(eigenvector_centrality_value)

df_betweenness_centrality.columns=['Station','Betweenness Centrality']
df_closeness_centrality.columns=['Station','Closeness Centrality']
df_eigenvector_centrality.columns=['Station','Eigenvector Centrality']

In [7]:
df_betweenness_centrality.head(10)

,Station,Betweenness Centrality
0,Stratford,0.098553
1,Liverpool Street,0.034307
2,Bank and Monument,0.027956
3,Canary Wharf,0.027956
4,Canning Town,0.027757
5,West Ham,0.024551
6,Highbury & Islington,0.023023
7,Whitechapel,0.019682
8,Canada Water,0.017898
9,Shadwell,0.017070


In [8]:
df_closeness_centrality.head(10)

,Station,Closeness Centrality
0,Stratford,0.927739
1,Highbury & Islington,0.836134
2,Whitechapel,0.820619
3,West Brompton,0.817248
4,Canada Water,0.813906
5,Bank and Monument,0.810591
6,Canary Wharf,0.810591
7,Richmond,0.810591
8,Canning Town,0.808943
9,Liverpool Street,0.808943


In [9]:
df_eigenvector_centrality.head(10)

,Station,Eigenvector Centrality
0,Stratford,0.073653
1,Canada Water,0.073103
2,Liverpool Street,0.073062
3,Canary Wharf,0.072862
4,Whitechapel,0.072779
5,Bank and Monument,0.072534
6,London Bridge,0.072491
7,Victoria,0.072491
8,Highbury & Islington,0.072480
9,Canning Town,0.072396


## I.2 Impact measures
Network connectivity

In [10]:
def cal_net_connect(G,nodes):
    original_avg_shortest_path = nx.average_shortest_path_length(G)

    # remove note
    G.remove_node(nodes)
    # Calculate the average shortest path after a node is removed
    removed_avg_shortest_path = nx.average_shortest_path_length(G)
    return original_avg_shortest_path,removed_avg_shortest_path

Modular metrics

In [11]:
def cal_net_connect_modularity1(data_G, node):
    
    if not nx.is_connected(data_G):
        partition = community.best_partition(data_G)
        original_modularity = community.modularity(partition, data_G)
        original_avg_shortest_path = float('inf')
        data_G.remove_node(node)
        if not nx.is_connected(data_G):
            partition = community.best_partition(data_G)
            removed_modularity = community.modularity(partition, data_G)
            removed_avg_shortest_path = float('inf')
        else:          
            removed_avg_shortest_path = nx.average_shortest_path_length(data_G)
    else:
        partition = community.best_partition(data_G)
        original_modularity = community.modularity(partition, data_G)
        original_avg_shortest_path = nx.average_shortest_path_length(data_G)
        # remove note
        data_G.remove_node(node)
        partition = community.best_partition(data_G)
        removed_modularity = community.modularity(partition, data_G)
        if not nx.is_connected(data_G):
#             original_avg_shortest_path = float('inf')
            removed_avg_shortest_path = float('inf')
        else:          
            # Calculate the average shortest path after a node is removed
            removed_avg_shortest_path = nx.average_shortest_path_length(data_G)
        
    return original_avg_shortest_path, removed_avg_shortest_path, original_modularity, removed_modularity

In [12]:
def cal_del_top10_net_connect_modularity(G,nodes_list):
    tmp_G = G.copy()
    all_list = list()
    tmp_dic = dict()
    for node in nodes_list:
        tmp_dic.clear()
        original_avg_shortest_path,removed_avg_shortest_path,original_modularity,removed_modularity = cal_net_connect_modularity1(tmp_G,node)
        tmp_dic['node'] = node
        tmp_dic['original_avg_shortest_path'] = original_avg_shortest_path
        tmp_dic['removed_avg_shortest_path'] = removed_avg_shortest_path
        tmp_dic['original_modularity'] = original_modularity
        tmp_dic['removed_modularity'] = removed_modularity
        all_list.append(tmp_dic.copy())
    return all_list

## I.3 Node removal
A) Non-sequential removal

In [13]:
del_top10_between =[i[0] for i in betweenness_centrality_value[:10]]
del_top10_close = [i[0] for i in closeness_centrality_value[:10]]
del_top10_eigenvector =[i[0] for i in eigenvector_centrality_value[:10]]

In [14]:
top10_between = cal_del_top10_net_connect_modularity(G,del_top10_between)
df_non_seq_between = pd.DataFrame(top10_between)

top10_close = cal_del_top10_net_connect_modularity(G,del_top10_close)
df_non_seq_close = pd.DataFrame(top10_close)

top10_eigenvector = cal_del_top10_net_connect_modularity(G,del_top10_eigenvector)
df_non_seq_close = pd.DataFrame(top10_eigenvector)

In [15]:
df_non_seq_between

,node,original_avg_shortest_path,removed_avg_shortest_path,original_modularity,removed_modularity
0,Stratford,1.612297,1.639976,0.163201,0.162965
1,Liverpool Street,1.639976,1.713037,0.163110,0.164182
2,Bank and Monument,1.713037,1.715318,0.164030,0.164422
3,Canary Wharf,1.715318,1.718113,0.164422,0.164888
4,Canning Town,1.718113,1.732437,0.164730,0.165074
5,West Ham,1.732437,1.855208,0.165074,0.165387
6,Highbury & Islington,1.855208,1.858382,0.165387,0.165393
7,Whitechapel,1.858382,1.861512,0.165393,0.166147
8,Canada Water,1.861512,1.877094,0.166147,0.166993
9,Shadwell,1.877094,inf,0.166764,0.164700


In [16]:
df_non_seq_close

,node,original_avg_shortest_path,removed_avg_shortest_path,original_modularity,removed_modularity
0,Stratford,1.612297,1.639976,0.163201,0.163110
1,Canada Water,1.639976,1.642076,0.162965,0.163851
2,Liverpool Street,1.642076,1.715369,0.163851,0.164919
3,Canary Wharf,1.715369,1.717664,0.164919,0.165266
4,Whitechapel,1.717664,1.723912,0.165266,0.166050
5,Bank and Monument,1.723912,1.726762,0.165861,0.166214
6,London Bridge,1.726762,1.728587,0.166214,0.167444
7,Victoria,1.728587,1.730422,0.167444,0.168873
8,Highbury & Islington,1.730422,1.740927,0.168688,0.168683
9,Canning Town,1.740927,1.755638,0.168683,0.169028


In [17]:
df_non_seq_close

,node,original_avg_shortest_path,removed_avg_shortest_path,original_modularity,removed_modularity
0,Stratford,1.612297,1.639976,0.163201,0.163110
1,Canada Water,1.639976,1.642076,0.162965,0.163851
2,Liverpool Street,1.642076,1.715369,0.163851,0.164919
3,Canary Wharf,1.715369,1.717664,0.164919,0.165266
4,Whitechapel,1.717664,1.723912,0.165266,0.166050
5,Bank and Monument,1.723912,1.726762,0.165861,0.166214
6,London Bridge,1.726762,1.728587,0.166214,0.167444
7,Victoria,1.728587,1.730422,0.167444,0.168873
8,Highbury & Islington,1.730422,1.740927,0.168688,0.168683
9,Canning Town,1.740927,1.755638,0.168683,0.169028


B) Sequential removal

In [21]:
def cal_del_top10_net_connect_modularity_sequential(G,node):
    tmp_dic = dict()
    original_avg_shortest_path,removed_avg_shortest_path,original_modularity,removed_modularity = cal_net_connect_modularity1(G,node)
    tmp_dic['node'] = node
    tmp_dic['original_avg_shortest_path'] = original_avg_shortest_path
    tmp_dic['removed_avg_shortest_path'] = removed_avg_shortest_path
    tmp_dic['original_modularity'] = original_modularity
    tmp_dic['removed_modularity'] = removed_modularity

    return tmp_dic


G1 =G.copy()
tmp1 = list()
for i in range(10):
    tmp_closeness_centrality_value1 = dict_sort_by_value(nx.closeness_centrality(G1))
    nodes_name1 = tmp_closeness_centrality_value1[0][0]
    all_dic1 = cal_del_top10_net_connect_modularity_sequential(G1,nodes_name1)
    tmp1.append(all_dic1)
df_seq_close = pd.DataFrame(tmp1)


G2 =G.copy()
tmp2 = list()
for i in range(10):
    tmp_closeness_centrality_value2 = dict_sort_by_value(nx.betweenness_centrality(G2))
    nodes_name2 = tmp_closeness_centrality_value2[0][0]
    all_dic2 = cal_del_top10_net_connect_modularity_sequential(G2,nodes_name2)
    tmp2.append(all_dic2)
df_seq_between = pd.DataFrame(tmp2)

G3 =G.copy()
tmp3 = list()
for i in range(10):
    tmp_closeness_centrality_value3 = dict_sort_by_value(nx.eigenvector_centrality(G3))
    nodes_name3 = tmp_closeness_centrality_value3[0][0]
    all_dic3 = cal_del_top10_net_connect_modularity_sequential(G3,nodes_name3)
    tmp3.append(all_dic3)
df_seq_eigenvector = pd.DataFrame(tmp3)

In [22]:
df_seq_between

,node,original_avg_shortest_path,removed_avg_shortest_path,original_modularity,removed_modularity
0,Stratford,1.612297,1.639976,0.163201,0.162965
1,Liverpool Street,1.639976,1.713037,0.162965,0.164030
2,Upminster,1.713037,inf,0.164030,0.164799
3,Bank and Monument,inf,inf,0.164799,0.165340
4,Canary Wharf,inf,inf,0.165196,0.165509
5,Canning Town,inf,inf,0.165657,0.165859
6,West Ham,inf,inf,0.165998,0.166177
7,Shadwell,inf,inf,0.166318,0.163498
8,Highbury & Islington,inf,inf,0.163675,0.163699
9,Whitechapel,inf,inf,0.163496,0.164154


In [23]:
df_seq_close

,node,original_avg_shortest_path,removed_avg_shortest_path,original_modularity,removed_modularity
0,Stratford,1.612297,1.639976,0.163201,0.163110
1,Highbury & Islington,1.639976,1.642317,0.162965,0.163029
2,Whitechapel,1.642317,1.644457,0.163029,0.163610
3,West Brompton,1.644457,1.646585,0.163795,0.164060
4,Canada Water,1.646585,1.660273,0.164060,0.164652
5,Bank and Monument,1.660273,1.662434,0.164652,0.165030
6,Canary Wharf,1.662434,1.665118,0.165030,0.165319
7,Richmond,1.665118,1.667493,0.165319,0.165300
8,Canning Town,1.667493,1.681985,0.165300,0.165857
9,Liverpool Street,1.681985,1.757573,0.165597,0.166625


In [24]:
df_seq_eigenvector

,node,original_avg_shortest_path,removed_avg_shortest_path,original_modularity,removed_modularity
0,Stratford,1.612297,1.639976,0.163326,0.163110
1,Canada Water,1.639976,1.642076,0.162965,0.163851
2,Liverpool Street,1.642076,1.715369,0.164010,0.165086
3,Canary Wharf,1.715369,1.717664,0.165086,0.165266
4,Whitechapel,1.717664,1.723912,0.165266,0.165861
5,London Bridge,1.723912,1.725723,0.165861,0.167075
6,Victoria,1.725723,1.727543,0.167075,0.168303
7,Paddington,1.727543,1.729359,0.168303,0.169540
8,Bank and Monument,1.729359,1.732252,0.169540,0.169941
9,Highbury & Islington,1.732252,1.742785,0.169941,0.169956


# II. Flows: Weighted Network
# II.1. Old vs new measure

In [25]:
weighted_betweenness = dict_sort_by_value(nx.betweenness_centrality(G, weight='flows'))

In [26]:
df_weighted_betweenness_centrality = pd.DataFrame(weighted_betweenness,columns=['nodes','weighted_betweenness_centrality'])

In [27]:
df_weighted_betweenness_centrality.head(10)

,nodes,weighted_betweenness_centrality
0,West Ham,1.227109e+89
1,West Brompton,7.276919e+88
2,Shepherd's Bush,3.576942e+88
3,Kew Gardens,2.073044e+88
4,Surrey Quays,1.047239e+88
5,Kenton,8.067562e+87
6,Richmond,7.703907e+87
7,Willesden Junction,2.793097e+87
8,Kentish Town West,1.796470e+87
9,Stratford,1.362334e+87


In [28]:
weighted_closeness = dict_sort_by_value(nx.closeness_centrality(G, distance='flows'))
df_weighted_closeness = pd.DataFrame(weighted_closeness,columns=['nodes','weighted_closeness_centrality'])
df_weighted_closeness.head(10)

,nodes,weighted_closeness_centrality
0,Abbey Road,5.605634
1,Bank and Monument,5.605634
2,Beckton,5.605634
3,Blackwall,5.605634
4,Canary Wharf,5.605634
5,Canning Town,5.605634
6,Crossharbour,5.605634
7,Custom House,5.605634
8,Cutty Sark,5.605634
9,Cyprus,5.605634


In [29]:
weighted_eigenvector =dict_sort_by_value(nx.eigenvector_centrality(G, weight='flows'))
df_weighted_eigenvector = pd.DataFrame(weighted_eigenvector,columns=['nodes','weighted_eigenvector_centrality'])
df_weighted_eigenvector.head(10)

,nodes,weighted_eigenvector_centrality
0,Waterloo,0.562523
1,Bank and Monument,0.494376
2,Canary Wharf,0.369500
3,Stratford,0.223366
4,London Bridge,0.201746
5,Liverpool Street,0.169696
6,King's Cross St. Pancras,0.119141
7,Oxford Circus,0.113297
8,Victoria,0.108028
9,Bond Street,0.105382


## II.2 Impact measure with flows

In [30]:
def cal_net_connect_modularity_weight(data_G, node):
    partition = community.best_partition(data_G, weight='flows')

    # Calculate the original network's modularity
    original_modularity = community.modularity(partition, data_G, weight='flows')
    
    if not nx.is_connected(data_G):
        original_avg_shortest_path = float('inf')
        removed_avg_shortest_path = float('inf')
    else:
        original_avg_shortest_path = nx.average_shortest_path_length(data_G, weight='flows')
        
        # Remove the node
        data_G.remove_node(node)
        # Re-calculate the community partition
        partition = community.best_partition(data_G, weight='flows')

        # Calculate the modularity after removing the node
        removed_modularity = community.modularity(partition, data_G, weight='flows')
        
        if not nx.is_connected(data_G):
            removed_avg_shortest_path = float('inf')
        else:          
            # Calculate the average shortest path after removing the node
            removed_avg_shortest_path = nx.average_shortest_path_length(data_G, weight='flows')
        
    return original_avg_shortest_path, removed_avg_shortest_path, original_modularity, removed_modularity


def cal_net_connect_modularity1(data_G, node):
    
    if not nx.is_connected(data_G):
        partition = community.best_partition(data_G)
        original_modularity = community.modularity(partition, data_G)
        original_avg_shortest_path = float('inf')
        data_G.remove_node(node)
        if not nx.is_connected(data_G):
            partition = community.best_partition(data_G)
            removed_modularity = community.modularity(partition, data_G)
            removed_avg_shortest_path = float('inf')
        else:          
            removed_avg_shortest_path = nx.average_shortest_path_length(data_G)
    else:
        partition = community.best_partition(data_G)
        original_modularity = community.modularity(partition, data_G)
        original_avg_shortest_path = nx.average_shortest_path_length(data_G)
        # remove note
        data_G.remove_node(node)
        partition = community.best_partition(data_G)
        removed_modularity = community.modularity(partition, data_G)
        if not nx.is_connected(data_G):
#             original_avg_shortest_path = float('inf')
            removed_avg_shortest_path = float('inf')
        else:          
            # Calculate the average shortest path after a node is removed
            removed_avg_shortest_path = nx.average_shortest_path_length(data_G)
        
    return original_avg_shortest_path, removed_avg_shortest_path, original_modularity, removed_modularity

In [31]:
def cal_del_top3_net_connect_modularity_weight(G,nodes_list):
    tmp_G = G.copy()
    all_list = list()
    tmp_dic = dict()
    for node in nodes_list:
        tmp_dic.clear()
#         print(r'remove node of {0}'.center(100,"*").format(node))
        original_avg_shortest_path,removed_avg_shortest_path,original_modularity,removed_modularity = cal_net_connect_modularity_weight(tmp_G,node)
        tmp_dic['node'] = node
        tmp_dic['original_avg_shortest_path'] = original_avg_shortest_path
        tmp_dic['removed_avg_shortest_path'] = removed_avg_shortest_path
        tmp_dic['original_modularity'] = original_modularity
        tmp_dic['removed_modularity'] = removed_modularity
        all_list.append(tmp_dic.copy())
    return all_list

## II.3  Experiment with flows

In [32]:
del_top3_close = [i[0] for i in closeness_centrality_value[:3]]
del_top3_between =[i[0] for i in betweenness_centrality_value[:3]]
del_top3_eigenvector =[i[0] for i in eigenvector_centrality_value[:3]]

In [33]:
top3_close = cal_del_top3_net_connect_modularity_weight(G,del_top3_close)
df_top3_close = pd.DataFrame(top3_close)
df_top3_close.head()

,node,original_avg_shortest_path,removed_avg_shortest_path,original_modularity,removed_modularity
0,Stratford,0.355121,0.356012,0.322133,0.332960
1,Highbury & Islington,0.356012,0.356907,0.320533,0.336802
2,Whitechapel,0.356907,0.357806,0.328636,0.331406


In [40]:
top3_between = cal_del_top3_net_connect_modularity_weight(G,del_top3_between)
df_top3_between = pd.DataFrame(top3_between)
df_top3_between.head()

,node,original_avg_shortest_path,removed_avg_shortest_path,original_modularity,removed_modularity
0,Stratford,0.355121,0.356012,0.328044,0.328676
1,Liverpool Street,0.356012,3.409498,0.332789,0.332306
2,Bank and Monument,3.409498,3.417875,0.338368,0.354231


In [41]:
top3_eigenvector = cal_del_top3_net_connect_modularity_weight(G,del_top3_eigenvector)
df_top3_eigenvector = pd.DataFrame(top3_eigenvector)
df_top3_eigenvector.head()

,node,original_avg_shortest_path,removed_avg_shortest_path,original_modularity,removed_modularity
0,Stratford,0.355121,0.356012,0.318798,0.328430
1,Canada Water,0.356012,0.356907,0.335961,0.326428
2,Liverpool Street,0.356907,3.417875,0.326219,0.322069


In [36]:
del_top3_close_weight = [i[0] for i in weighted_closeness[:3]]
del_top3_between_weight =[i[0] for i in weighted_betweenness[:3]]
del_top3_eigenvector_weight =[i[0] for i in weighted_eigenvector[:3]]

In [38]:
top3_between_weight = cal_del_top3_net_connect_modularity_weight(G,del_top3_between_weight)
df_top3_between_weight = pd.DataFrame(top3_between_weight)
df_top3_between_weight.head()

,node,original_avg_shortest_path,removed_avg_shortest_path,original_modularity,removed_modularity
0,West Ham,0.355121,0.356012,0.330529,0.328043
1,West Brompton,0.356012,0.356907,0.325453,0.323298
2,Shepherd's Bush,0.356907,0.357806,0.324685,0.331723


In [42]:
top3_close_weight = cal_del_top3_net_connect_modularity_weight(G,del_top3_close_weight)
df_top3_close_weight = pd.DataFrame(top3_close_weight)
df_top3_close_weight.head()

,node,original_avg_shortest_path,removed_avg_shortest_path,original_modularity,removed_modularity
0,Abbey Road,0.355121,0.356012,0.321799,0.324697
1,Bank and Monument,0.356012,0.356907,0.314477,0.334143
2,Beckton,0.356907,0.357806,0.340400,0.340057


In [43]:
top3_eigenvector_weight = cal_del_top3_net_connect_modularity_weight(G,del_top3_eigenvector_weight)
df_top3_eigenvector_weight = pd.DataFrame(top3_eigenvector_weight)
df_top3_eigenvector_weight.head()

,node,original_avg_shortest_path,removed_avg_shortest_path,original_modularity,removed_modularity
0,Waterloo,0.355121,0.356012,0.327117,0.333625
1,Bank and Monument,0.356012,0.356907,0.328080,0.343629
2,Canary Wharf,0.356907,0.357806,0.344994,0.360908


# III. Models and calibration 
## III.1. Spatial interaction models


In [41]:
def predict_flows_function(beta):
    all_predicted_flows = list()
    total_predicted_flows = 0
    total_t = 0
    for u, v, data in G.edges(data=True):
        flows = data['flows']
        population = data['population']
        jobs = data['jobs']
        distance = data['distance']
        
        predicted_flows = population * jobs * np.exp(-1*beta*distance)
        all_predicted_flows.append(predicted_flows)
        total_t+=flows
        total_predicted_flows+=predicted_flows
        
    end_rest = (total_t*np.array(all_predicted_flows))/total_predicted_flows
    
    return end_rest

from scipy.optimize import minimize

def cost_function(beta):
    tmp_all_predicted_flows = list()
    all_observed_flows = list()
    total_predicted_flows = 0
    total_t = 0
    total_cost = 0
    for u, v, data in G.edges(data=True):
        flows = data['flows']
        population = data['population']
        jobs = data['jobs']
        distance = data['distance']
        
        predicted_flows = population * jobs * np.exp(-1*beta*distance)
        tmp_all_predicted_flows.append(predicted_flows)
        all_observed_flows.append(flows)
        total_t+=flows
        total_predicted_flows+=predicted_flows
        
    all_predicted_flows = (total_t*np.array(tmp_all_predicted_flows))/total_predicted_flows       
    total_cost = np.sum((np.array(all_observed_flows)-all_predicted_flows)**2)
    return total_cost



def cost_function1(beta):
    total_cost = 0
    for u, v, data in G.edges(data=True):
        flows = data['flows']
        population = data['population']
        jobs = data['jobs']
        distance = data['distance']
        
        predicted_flows = beta * (population * jobs) / (distance+0.001)
        cost = (predicted_flows - flows) ** 2
        total_cost += cost
    
    return total_cost

In [42]:
initial_beta = 0.5
beta = initial_beta
observed_flows = np.array([data['flows'] for u, v, data in G.edges(data=True)])


res = minimize(cost_function, beta)
new_beta = res.x[0]
all_predicted_flows = predict_flows_function(new_beta)

MSE  = mean_squared_error(observed_flows,all_predicted_flows)
MAE  = mean_absolute_error(observed_flows,all_predicted_flows)
RMSE = np.sqrt(mean_squared_error(observed_flows,all_predicted_flows))
R2   = r2_score(observed_flows,all_predicted_flows)



beta = new_beta
print('MSE:{}'.format(MSE))
print('MAE:{}'.format(MAE))
print('RMSE:{}'.format(RMSE))
print('R2:{}'.format(R2))

print("Optimized beta:", beta)

MSE:3110367.870059292
MAE:50.96910271769973
RMSE:1763.623505757193
R2:-130.69421586194665
Optimized beta: 0.5


# IV. Scenarios
## IV.1. Scenario A

In [43]:
original_jobs_canary_wharf = df.loc[df['station_destination']=='Canary Wharf']['jobs'].values[0]
df.loc[df['station_destination']=='Canary Wharf','jobs'] = original_jobs_canary_wharf/2
G = nx.from_pandas_edgelist(df, 'station_origin', "station_destination", ["flows", "population",'jobs','distance'])

In [44]:
def predict_flows_function(beta):
    all_predicted_flows = list()
    total_predicted_flows = 0
    total_t = 0
    for u, v, data in G.edges(data=True):
        flows = data['flows']
        population = data['population']
        jobs = data['jobs']
        distance = data['distance']
        
        predicted_flows = population * jobs * np.exp(-1*beta*distance)
        all_predicted_flows.append(predicted_flows)
        total_t+=flows
        total_predicted_flows+=predicted_flows
        
    end_rest = (total_t*np.array(all_predicted_flows))/total_predicted_flows
    
    return end_rest


def cost_function(beta):
    tmp_all_predicted_flows = []
    all_observed_flows = []
    total_predicted_flows = 0
    total_t = 0
    total_cost = 0
    
    for u, v, data in G.edges(data=True):
        flows = data['flows']
        population = data['population']
        jobs = data['jobs']
        distance = data['distance']
        
        predicted_flows = population * jobs * np.exp(-1 * beta * distance)
        tmp_all_predicted_flows.append(predicted_flows)
        all_observed_flows.append(flows)
        total_t += flows
        total_predicted_flows += predicted_flows
        
    all_predicted_flows = (total_t * np.array(tmp_all_predicted_flows)) / total_predicted_flows
    total_cost = np.sum((np.array(all_observed_flows) - all_predicted_flows) ** 2)
    
    return total_cost

In [53]:
initial_beta = 0.5

res = minimize(cost_function, [initial_beta], method='BFGS')
optimized_beta = res.x[0]
all_predicted_flows = predict_flows_function(optimized_beta)


total_observed_flows = np.array([data['flows'] for u, v, data in G.edges(data=True)])
scaling_factor = np.sum(total_observed_flows) / np.sum(all_predicted_flows)
MSE  = mean_squared_error(observed_flows,all_predicted_flows)
MAE  = mean_absolute_error(observed_flows,all_predicted_flows)
RMSE = np.sqrt(mean_squared_error(observed_flows,all_predicted_flows))
R2   = r2_score(observed_flows,all_predicted_flows)
print('MSE:{}'.format(MSE))
print('MAE:{}'.format(MAE))
print('RMSE:{}'.format(RMSE))
print('R2:{}'.format(R2))
print('predict:observe={}'.format(scaling_factor))

ValueError: not enough values to unpack (expected 3, got 1)

In [51]:
def predict_flows_function(beta,param1,param2):
    all_predicted_flows = list()
    total_predicted_flows = 0
    total_t = 0
    for u, v, data in G.edges(data=True):
        flows = data['flows']
        population = data['population']
        jobs = data['jobs']
        distance = data['distance']
        
        predicted_flows = population * jobs * np.exp(-1*beta*distance*param1)+param2
        all_predicted_flows.append(predicted_flows)
        total_t+=flows
        total_predicted_flows+=predicted_flows
        
    end_rest = (total_t*np.array(all_predicted_flows))/total_predicted_flows
    
    return end_rest


def cost_function(params):
    beta, param1, param2 = params
    tmp_all_predicted_flows = []
    all_observed_flows = []
    total_predicted_flows = 0
    total_t = 0
    total_cost = 0
    
    for u, v, data in G.edges(data=True):
        flows = data['flows']
        population = data['population']
        jobs = data['jobs']
        distance = data['distance']
        
        predicted_flows = population * jobs * np.exp(-1 * beta * distance * param1) + param2
        tmp_all_predicted_flows.append(predicted_flows)
        all_observed_flows.append(flows)
        total_t += flows
        total_predicted_flows += predicted_flows
        
    all_predicted_flows = (total_t * np.array(tmp_all_predicted_flows)) / total_predicted_flows
    total_cost = np.sum((np.array(all_observed_flows) - all_predicted_flows) ** 2)
    
    return total_cost

In [52]:
initial_beta = 0.5
initial_param1 = 1.0
initial_param2 = 0.0


initial_params = [0.5, 1.0, 0.0]


res = minimize(cost_function, x0=initial_params, args=(), method='BFGS', options={'disp': True})
optimized_params = res.x


optimized_beta = optimized_params[0]
optimized_param1 = optimized_params[1]
optimized_param2 = optimized_params[2]

all_predicted_flows = predict_flows_function(optimized_beta,optimized_param1,optimized_param2)


total_observed_flows = np.array([data['flows'] for u, v, data in G.edges(data=True)])
scaling_factor = np.sum(total_observed_flows) / np.sum(all_predicted_flows)
MSE  = mean_squared_error(observed_flows,all_predicted_flows)
MAE  = mean_absolute_error(observed_flows,all_predicted_flows)
RMSE = np.sqrt(mean_squared_error(observed_flows,all_predicted_flows))
R2   = r2_score(observed_flows,all_predicted_flows)
print('MSE:{}'.format(MSE))
print('MAE:{}'.format(MAE))
print('RMSE:{}'.format(RMSE))
print('R2:{}'.format(R2))
print('predict:observe={}'.format(scaling_factor))

         Current function value: 1913126279.995694
         Iterations: 3
         Function evaluations: 220
         Gradient evaluations: 52
MSE:60316.73749907606
MAE:36.906647312419
RMSE:245.5946609742892
R2:-1.553834716708372
predict:observe=1.000000000000235
